# Selenium 구글 이미지 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/ /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=chrome_options)

     |████████████████████████████████| 911kB 5.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.

NameError: ignored

## 필요 라이브러리 import

In [8]:
import os
import time
import socket

from urllib.request import urlretrieve
from urllib.error import HTTPError, URLError
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException, ElementNotInteractableException
from PIL import Image

## `scroll_down()`: 스크롤을 내리는 함수

In [55]:
# 스크롤 다운 : 스크롤을 내리는 함수
def scroll_down():
    scroll_count = 0
    print("[scroll_down(): 스크롤 다운 시작]")
    
    last_height = wd.execute_script("return document.body.scrollHeight")
    after_click = False

    while True:
        print(f"[스크롤 다운: {scroll_count}]")
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        scroll_count += 1
        time.sleep(1)

        new_height = wd.execute_script("return document.body.scrollHeight")

        if last_height == new_height:
            if after_click is True:
                break
            else:
                try:
                    more_button = wd.find_element_by_xpath("//*[@id='islmp']/div/div/div/div/div[5]/input")
                    if more_button.is_displayed():
                        more_button.click()
                        after_click = True
                except NoSuchElementException as e:
                    print(e)
                    break
        last_height = new_height


## `click_and_save()`: 썸네일 이미지 선택 후 원본 이미지 저장

In [56]:
# 클릭하고 저장하는 함수
def click_and_save(dir_name, index, img, img_list_length):
    global scraped_count

    try:
        img.click()
        wd.implicitly_wait(3)
        src = wd.find_element_by_xpath('//*[@id="Sva75c"]/div/div/div[3]/div[2]/c-wiz/div[1]/div[1]/div/div[2]/a/img').get_attribute('src')

        if src.split('.')[-1] == 'png' :
            urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.png')
            print(f"{index + 1}/{img_list_length} PNG 이미지 저장")
        else:
            urlretrieve(src, dir_name + '/' + str(scraped_count + 1) + '.jpg')
            print(f"{index + 1}/{img_list_length} JPG 이미지 저장")

        scraped_count += 1

    except HTTPError as e:
        print(e)
        pass

    except ElementClickInterceptedException as e:
        print(e)
        pass

## `scraping()`: 구글 이미지 스크래핑 시작

In [ ]:
# 구글 이미지 스크래핑 함수
def scraping(dir_name, query):
    global scaraped_count

    url = f"https://www.google.co.kr/search?q={query}sea&tbm=isch&ved=2ahUKEwiEk8P-hpjrAhVNbJQKHZAbDBUQ2-cCegQIABAA&oq=sea&gs_lcp=CgNpbWcQAzIFCAAQsQMyBQgAELEDMgUIABCxAzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6BAgjECdQ_OkCWPHtAmCw8QJoAHAAeACAAW2IAcECkgEDMC4zmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=ESE1X8TGDM3Y0QSQt7CoAQ&bih=865&biw=857&tbs=isz%3Al&hl=ko&hl=ko"
    print(url)
    wd.get(url)
    wd.maximize_window()

    scroll_down()

    div = wd.find_element_by_xpath('//*[@id="islrg"]/div[1]')
    img_list = div.find_elements_by_css_selector('div.bRMDJf.islir > img')
    print(img_list)
    for index, img in enumerate(img_list):
        print(index)
        try:
            click_and_save(dir_name, index, img, len(img_list))

        except ElementClickInterceptedException as e:
            print(e)
            wd.execute_script("window.scrollTo(0, window.scrollY + 100")
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))

        except NoSuchElementException as e:
            print(e)
            wd.execute_script("window.scrollTo(0, window.scrollY + 100")
            time.sleep(1)
            click_and_save(dir_name, index, img, len(img_list))
        except ConnectionResetError as e:
            print(e)
            pass # 무시한다는 뜻
        except URLError as e:
            print(e)
            pass
        except socket.timeout as e:
            print(e)
            pass
        except socket.gaierror as e:
            print(e)
            pass
        except ElementNotInteractableException as e:
            print(e)
            pass

    try:
        print("[스크래핑 종료 (성공률 : %.2f%%)]" % (scraped_count / len(img_list) * 100.0))
    except ZeroDivisionError as e:
        print(e)

    wd.quit()


## `filter_and_remove()`: 일정 해상도 이하이거나 손상된 이미지 제거

In [50]:
def filter_and_remove(dir_name, query, filter_size):
    filtered_size = 0

    for index, file_name in enumerate(os.listdir(dir_name)):
        try:

            file_path = os.path.join(dir_name, file_name)
            img = Image.open(file_path)

            if img.width < filter_size and img.height < filter_size:
                img.close()
                os.remove(file_path)
                print(f"{index} 이미지 제거")
                filtered_count += 1
        
        except OSError as e:
            print(e)
            os.remove(file_path)
            filtered_count += 1

    print(f"이미지 제거 개수: {filtered_count}/{scraped_count}]")


## 이미지 확인

In [57]:
socket.setdefaulttimeout(30)

wd = webdriver.Chrome('chromedriver', options = chrome_options)
scraped_count = 0

path = "./"
query = input("검색어 입력: ")

dir_name = path + query
os.makedirs(dir_name)
print(f"[{dir_name} 디렉토리 생성]")

scraping(dir_name, query)
filter_and_remove(dir_name, query, 400 )

검색어 입력: forest
[./forest 디렉토리 생성]
https://www.google.co.kr/search?q=forestsea&tbm=isch&ved=2ahUKEwiEk8P-hpjrAhVNbJQKHZAbDBUQ2-cCegQIABAA&oq=sea&gs_lcp=CgNpbWcQAzIFCAAQsQMyBQgAELEDMgUIABCxAzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6BAgjECdQ_OkCWPHtAmCw8QJoAHAAeACAAW2IAcECkgEDMC4zmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=ESE1X8TGDM3Y0QSQt7CoAQ&bih=865&biw=857&tbs=isz%3Al&hl=ko&hl=ko
[scroll_down(): 스크롤 다운 시작]
[스크롤 다운: 0]
[스크롤 다운: 1]
[스크롤 다운: 2]
[스크롤 다운: 3]
[스크롤 다운: 4]
[스크롤 다운: 5]
[스크롤 다운: 6]
[스크롤 다운: 7]
[스크롤 다운: 8]
[스크롤 다운: 9]
[스크롤 다운: 10]
[<selenium.webdriver.remote.webelement.WebElement (session="ed269b9add45b494f22d2c481343a82c", element="e06c8004-5a1e-484b-8da5-53daf50cf6e0")>, <selenium.webdriver.remote.webelement.WebElement (session="ed269b9add45b494f22d2c481343a82c", element="c384ae20-2d3e-434d-bf00-a6c3b0d02a6d")>, <selenium.webdriver.remote.webelement.WebElement (session="ed269b9add45b494f22d2c481343a82c", element="6a5c14db-7071-4e71-b15a-dec9d973c629")>, <selenium.webdriver.re

UnboundLocalError: ignored

In [ ]:
!rmdir sea

In [58]:
!ls forest # forest 이미지 폴더 내부보기

100.jpg  16.jpg   238.jpg  306.jpg  375.jpg  443.jpg  512.jpg  581.jpg	64.jpg
101.jpg  170.jpg  239.jpg  307.jpg  376.jpg  444.jpg  513.jpg  582.jpg	650.jpg
102.jpg  171.jpg  23.jpg   308.jpg  377.jpg  445.jpg  514.jpg  583.jpg	651.jpg
103.jpg  172.jpg  240.jpg  309.jpg  378.jpg  446.jpg  515.jpg  584.jpg	652.jpg
104.jpg  173.jpg  241.jpg  30.jpg   379.jpg  447.jpg  516.jpg  585.jpg	653.jpg
105.jpg  174.jpg  242.jpg  310.jpg  37.jpg   448.jpg  517.jpg  586.jpg	654.jpg
106.jpg  175.jpg  243.jpg  311.jpg  380.jpg  449.jpg  518.jpg  587.jpg	655.jpg
107.jpg  176.jpg  244.jpg  312.jpg  381.jpg  44.jpg   519.jpg  588.jpg	656.jpg
108.jpg  177.jpg  245.jpg  313.jpg  382.jpg  450.jpg  51.jpg   589.jpg	657.jpg
109.jpg  178.jpg  246.jpg  314.jpg  383.jpg  451.jpg  520.jpg  58.jpg	658.jpg
10.jpg	 179.jpg  247.jpg  315.jpg  384.jpg  452.jpg  521.jpg  590.jpg	659.jpg
110.jpg  17.jpg   248.jpg  316.jpg  385.jpg  453.jpg  522.jpg  591.jpg	65.jpg
111.jpg  180.jpg  249.jpg  317.jpg  386.jpg  454.jpg  52

In [59]:
# 다운로드해서 보기
from google.colab import files
files.download("./forest/14.jpg")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 요즘에는 딥러닝 학습목적으로 스크래핑하여 이미지 수집 - 라벨링 등으로 사용